In [2]:
#Biblioteca para a mineração de dados
from bs4 import BeautifulSoup

#Biblioteca para lidar com o HTTP 
import requests

#Biblioteca para exportar para csv
from csv import writer

url = 'https://www.pararius.com/apartments/amsterdam?ac=1'
page = requests.get(url)
#Aqui é importante dar um print(page) para ver se o request é 200, ou seja, se deu certo o request para a página

soup = BeautifulSoup(page.content, 'html.parser')
lists = soup.find_all('section')

with open('casas.csv', 'w', encoding='utf8', newline='') as f:
    thewriter = writer(f)
    header = ['Title', 'Location', 'Price', 'Size', 'Area']
    thewriter.writerow(header)
    for list in lists:
        title = list.find('a', class_ = 'listing-search-item__link--title').text.replace('\n', '')
        location = list.find('div', class_ = 'listing-search-item__sub-title').text.replace('\n', '')
        price = list.find('div', class_ = 'listing-search-item__price').text.replace('\n', '')
        size = list.find('ul', class_ = 'illustrated-features').text.replace('\n', '')
        area = list.find('a', class_ = 'listing-search-item__link').text.replace('\n', '')
        info = [title, location, price, size, area]
        thewriter.writerow(info)